# Import & Load Childhood data (Under-One Year)

### Config

In [ ]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../../settings.ini"))

sys.path.insert(0, config.get("PATHS", "libs_path"))
engine_path = config.get("DATABASE", "engine_path")

### Main params

In [ ]:
remote_path = ""
local_path = "../data_final/"

In [ ]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [ ]:
#Open unzipped file
df = pd.read_csv(local_path + "ive_junaeb.csv", delimiter=",")
# df = df[['year','comuna_id','sex_id','rate_comuna','rate_region','rate_country','count','age_range_id']]
# df = df.astype({"year":"int", "comuna_id": "int",'sex_id':'int','rate_comuna':'float','rate_region':'float','rate_country':'float', 'age_range_id': 'float'})
list(df)

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, "childhood", "fact_vulnerability")

### Add Indices to foreign keys

In [ ]:
engine.execute("""
CREATE INDEX fact_vulnerability_comuna_id_index 
ON childhood.fact_vulnerability (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_vulnerability_year_index 
ON childhood.fact_vulnerability ("year");
""")

In [ ]:
d2 = download_file(remote_path, local_path, "priority.csv")
print (inline_table_xml(d2, "name_en", "id", "name_es"))